In [1]:
import os
import cv2
import torch
from torch.optim import SGD
from torch.utils.data import DataLoader
import segmentation_models_pytorch as sm
import cv2
import catalyst
import wandb
from PIL import Image
import albumentations
from sklearn.model_selection import train_test_split
import numpy as np
from collections import defaultdict
import copy
import time
from tqdm.auto import tqdm
import torch.nn.functional as F
import sklearn
import pickle
import matplotlib.pyplot as plt
import random

from utils.masks import getData, augmentation

In [ ]:
!cp -r 'logs/FloodNetData/FloodNetData/FloodNet/' 'Data/'

In [2]:
floodDir = "Data/FloodNet/Train/Labeled/Flooded"
nonFloodDir = "Data/FloodNet/Train/Labeled/Non-Flooded"

floodImgDir = os.path.join(floodDir, "image")
floodMaskDir = os.path.join(floodDir, "mask")
nonFloodImgDir = os.path.join(nonFloodDir, "image")
nonFloodMaskDir = os.path.join(nonFloodDir, "mask")

floodImg = len(os.listdir(floodImgDir))
floodMask = len(os.listdir(floodMaskDir))
nonFloodImg = len(os.listdir(nonFloodImgDir))
nonFloodMask = len(os.listdir(nonFloodMaskDir))
print(f"Flooded images:      {floodImg} Flooded masks:     {floodMask}")
print(f"Non-Flooded images: {nonFloodImg} Non-Flooded masks: {nonFloodMask}")

Flooded images:      204 Flooded masks:     204
Non-Flooded images: 1388 Non-Flooded masks: 1388


In [3]:
x =  [os.path.join(nonFloodImgDir, file) for file in sorted(os.listdir(nonFloodImgDir))]

In [5]:
dataset = getData(x)

## Augmentation

In [6]:
for image, mask, index in dataset:
    
    im1, im2, im3, m1, m2, m3 = augmentation(image, mask)
    
    idx = x[index].split('.')[0]
    diri1 = idx + '_0.jpg'
    dirm1 = diri1.replace('image','mask').replace('.jpg','_lab.png')
    
    im1.save(diri1)
    m1.save(dirm1)
    
    diri2 = idx + '_1.jpg'
    dirm2 = diri2.replace('image','mask').replace('.jpg','_lab.png')
    
    im2.save(diri2)
    m2.save(dirm2)
   
    diri3 = idx + '_2.jpg'
    dirm3 = diri3.replace('image','mask').replace('.jpg','_lab.png')
    
    im3.save(diri3)
    m3.save(dirm3)


In [ ]:
imgDim = 224
for i in (random.sample(x, 3)):
    
    idx = i.split('.')[0]
    idx1 = idx + '_0.jpg'
    idx2 = idx + '_1.jpg'
    idx3 = idx + '_2.jpg'
    
    original = cv2.imread(i)
    original = cv2.resize(original,(imgDim, imgDim))
    original_mask = cv2.imread(i.replace("image", "mask").replace(".jpg", "_lab.png"))
    original_mask = cv2.resize(original_mask,(imgDim, imgDim))
    original_mask = cv2.cvtColor(original_mask, cv2.COLOR_BGR2GRAY)
    
    im1 = cv2.imread(idx1)
    im1 = cv2.resize(im1,(imgDim, imgDim))
    m1 = cv2.imread(idx1.replace("image", "mask").replace(".jpg", "_lab.png"))
    m1 = cv2.resize(m1,(imgDim, imgDim))
    m1 = cv2.cvtColor(m1, cv2.COLOR_BGR2GRAY)
    
    im2 = cv2.imread(idx2)
    im2 = cv2.resize(im2,(imgDim, imgDim))
    m2 = cv2.imread(idx2.replace("image", "mask").replace(".jpg", "_lab.png"))
    m2 = cv2.resize(m2,(imgDim, imgDim))
    m2 = cv2.cvtColor(m2, cv2.COLOR_BGR2GRAY)
    
    im3 = cv2.imread(idx3)
    im3 = cv2.resize(im3,(imgDim, imgDim))
    m3 = cv2.imread(idx3.replace("image", "mask").replace(".jpg", "_lab.png"))
    m3 = cv2.resize(m3,(imgDim, imgDim))
    m3 = cv2.cvtColor(m3, cv2.COLOR_BGR2GRAY)


    figure, ax = plt.subplots(nrows = 4, ncols = 2, figsize=(20, 10))
    ax[0][0].imshow(original)
    ax[0][1].imshow(original_mask)
    ax[1][0].imshow(im1)
    ax[1][1].imshow(m1)
    ax[2][0].imshow(im2)
    ax[2][1].imshow(m2)
    ax[3][0].imshow(im3)
    ax[3][1].imshow(m3)
 
    figure.tight_layout()
    figure.show()

## New classes

{'Background':0,'Building':1,'Road':2, 'Water': 3,'Tree':4,'Vehicle':5,'Pool':6,'Grass':7}

In [ ]:
for image, mask, index in dataset:
    
    mask = np.array(mask)
    mask = np.ma.masked_where(mask == 2, mask)
    mask = mask.filled(1)
    mask = np.ma.masked_where(mask == 3, mask)
    mask = mask.filled(2)
    mask = np.ma.masked_where(mask == 4, mask)
    mask = mask.filled(2)
    mask = np.ma.masked_where(mask == 5, mask)
    mask = mask.filled(3)
    mask = np.ma.masked_where(mask == 6, mask)
    mask = mask.filled(4)
    mask = np.ma.masked_where(mask == 7, mask)
    mask = mask.filled(5)
    mask = np.ma.masked_where(mask == 8, mask)
    mask = mask.filled(6)
    mask = np.ma.masked_where(mask == 9, mask)
    mask = mask.filled(7)
    
    mask = Image.fromarray(mask)
    idx = x[index].replace('image','mask').replace('.jpg','_lab.png')
    mask.save(idx)